In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df=pd.read_csv('/content/drive/MyDrive/Customer Satisfaction/customer_capstone_project.csv')

In [4]:
# df.review_cleaned_text[3]
# df.review_sentiment[3]
df.duplicated().sum()

0

In [5]:
## Convert Sentiment Scores to Categories

# Function to convert sentiment score to categories
def convert_sentiment(score):
    if score <= -0.1:
        return 'negative'
    elif -0.1 < score <= 0.1:
        return 'neutral'
    else:
        return 'positive'

In [6]:
# Apply the function to the sentiment_score column
df['sentiment_label'] = df['review_sentiment'].apply(convert_sentiment)

# Check the result
df_sentiment = df[['review_cleaned_text', 'sentiment_label']]
df_sentiment.tail()

,review_cleaned_text,sentiment_label
10335,i ate here a few times before and was very ple...,positive
10336,best food ever have been trying hard to find ...,positive
10337,best food ever have been trying hard to find ...,positive
10338,awesome dive bar citywides here looooove love...,positive
10339,i had the crispy chicken wrap with dijon and f...,positive


In [7]:
df_sentiment

,review_cleaned_text,sentiment_label
0,good ambience\n\ngood service\n\nordered \n\nf...,positive
1,good ambience\n\ngood service\n\nordered \n\nf...,positive
2,good ambience\n\ngood service\n\nordered \n\nf...,positive
3,good ambience\n\ngood service\n\nordered \n\nf...,positive
4,good ambience\n\ngood service\n\nordered \n\nf...,positive
...,...,...
10335,i ate here a few times before and was very ple...,positive
10336,best food ever have been trying hard to find ...,positive
10337,best food ever have been trying hard to find ...,positive
10338,awesome dive bar citywides here looooove love...,positive


In [8]:
# Import all the neccessary libraries

from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

In [9]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Customer Satisfaction/vocab.txt')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2107: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [10]:
train_df, test_df = train_test_split(df_sentiment, test_size=0.2, random_state=42)

In [11]:
# Tokenize the text
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128)

In [12]:
# Apply tokenization
train_encodings = tokenizer(list(train_df['review_cleaned_text']), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_df['review_cleaned_text']), padding=True, truncation=True, max_length=128)

In [13]:
import torch
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convert data to PyTorch datasets
train_labels = [1 if label == 'positive' else 0 for label in train_df['sentiment_label']]  # Binary labels
test_labels = [1 if label == 'positive' else 0 for label in test_df['sentiment_label']]

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [14]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # Output directory
    num_train_epochs=3,               # Number of training epochs
    per_device_train_batch_size=8,    # Batch size per device during training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    warmup_steps=500,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # Strength of weight decay
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",      # Evaluation strategy to use
    save_strategy="epoch",            # Save strategy must match the evaluation strategy
    load_best_model_at_end=True,      # Load the best model when finished training
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.077900,0.160727
2,0.001600,0.100593
3,0.000400,0.117927


TrainOutput(global_step=3102, training_loss=0.12360960574724413, metrics={'train_runtime': 1591.1968, 'train_samples_per_second': 15.596, 'train_steps_per_second': 1.949, 'total_flos': 1632340987453440.0, 'train_loss': 0.12360960574724413, 'epoch': 3.0})

In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Evaluate the model
predictions, labels, _ = trainer.predict(test_dataset)

# Convert logits to predicted labels
predicted_labels = predictions.argmax(axis=-1)

# Calculate metrics
accuracy = accuracy_score(labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(labels, predicted_labels, average='binary')

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Accuracy: 0.9797
Precision: 0.9802
Recall: 0.9953
F1-Score: 0.9877


In [16]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [17]:
# Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the text for RoBERTa input
train_encodings = tokenizer(list(train_df['review_cleaned_text']), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_df['review_cleaned_text']), padding=True, truncation=True, max_length=128)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [18]:
# Convert labels to binary (positive=1, negative=0)
train_labels = [1 if label == 'positive' else 0 for label in train_df['sentiment_label']]  # Binary labels
test_labels = [1 if label == 'positive' else 0 for label in test_df['sentiment_label']]

In [19]:
# Convert data to PyTorch datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [20]:
# Load pre-trained RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # Output directory
    num_train_epochs=3,               # Number of training epochs
    per_device_train_batch_size=8,    # Batch size per device during training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    warmup_steps=500,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # Strength of weight decay
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",      # Evaluate every epoch
    save_strategy="epoch",            # Save model every epoch
    load_best_model_at_end=True,      # Load the best model when finished training
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-tune the model
trainer.train()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.445600,0.499463
2,0.376500,0.481870
3,0.429600,0.320500


TrainOutput(global_step=3102, training_loss=0.42265739311601636, metrics={'train_runtime': 741.3369, 'train_samples_per_second': 33.475, 'train_steps_per_second': 4.184, 'total_flos': 1632340987453440.0, 'train_loss': 0.42265739311601636, 'epoch': 3.0})

In [21]:
# Evaluate the model
predictions, labels, _ = trainer.predict(test_dataset)

# Convert logits to predicted labels
predicted_labels = predictions.argmax(axis=-1)

# Calculate metrics
accuracy = accuracy_score(labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(labels, predicted_labels, average='binary')

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Accuracy: 0.9047
Precision: 0.9129
Recall: 0.9770
F1-Score: 0.9438


In [26]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

In [24]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [27]:
# Define a function to preprocess the data
def preprocess_data(df):
    return [
        {
            'input_text': f"Classify sentiment: {text}",
            'target_text': 'positive' if sentiment == 'positive' else 'negative'
        }
        for text, sentiment in zip(df['review_cleaned_text'], df['sentiment_label'])
    ]

# Preprocess the data
train_data = preprocess_data(train_df)
test_data = preprocess_data(test_df)

# Convert to datasets
train_dataset = Dataset.from_pandas(pd.DataFrame(train_data))
test_dataset = Dataset.from_pandas(pd.DataFrame(test_data))

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Tokenize the data
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input_text'], padding=True, truncation=True, max_length=128)
    labels = tokenizer(examples['target_text'], padding=True, truncation=True, max_length=32)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small',num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/8272 [00:00<?, ? examples/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.114600,0.108121
2,0.082900,0.093919
3,0.063400,0.090584


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3102, training_loss=0.12939201548051865, metrics={'train_runtime': 353.0881, 'train_samples_per_second': 70.283, 'train_steps_per_second': 8.785, 'total_flos': 839660536332288.0, 'train_loss': 0.12939201548051865, 'epoch': 3.0})

In [28]:
from sklearn.metrics import accuracy_score

output = trainer.predict(test_dataset)

# Access the predictions (logits) from the output
logits = output.predictions

# Check the structure of logits if it is a tuple
# If logits are a tuple, extract the actual logits tensor
if isinstance(logits, tuple):
    logits = logits[0]

# Apply argmax to the logits to get the predicted class labels
predicted_labels = logits.argmax(axis=-1)  # Use 'axis=-1' to get the predicted class


# Flatten the labels and predicted labels if they are multi-output/multilabel
predicted_labels_flat = predicted_labels.flatten()
true_labels_flat = output.label_ids.flatten()

# Now calculate the metrics
accuracy = accuracy_score(true_labels_flat, predicted_labels_flat)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels_flat, predicted_labels_flat)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision.mean():.4f}")
print(f"Recall: {recall.mean():.4f}")
print(f"F1-Score: {f1.mean():.4f}")


Accuracy: 0.9785
Precision: 0.9628
Recall: 0.9380
F1-Score: 0.9495


In [29]:
df_sen=pd.read_csv('/content/drive/MyDrive/Customer Satisfaction/df_sentiment.csv')

In [30]:
df_sen

,review_cleaned_text,sentiment_label
0,good ambience\n\ngood service\n\nordered \n\nf...,positive
1,good ambience\n\ngood service\n\nordered \n\nf...,positive
2,good ambience\n\ngood service\n\nordered \n\nf...,positive
3,good ambience\n\ngood service\n\nordered \n\nf...,positive
4,good ambience\n\ngood service\n\nordered \n\nf...,positive
...,...,...
10335,i ate here a few times before and was very ple...,positive
10336,best food ever have been trying hard to find ...,positive
10337,best food ever have been trying hard to find ...,positive
10338,awesome dive bar citywides here looooove love...,positive


In [31]:
import pandas as pd
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load Spacy for lemmatization
nlp = spacy.load('en_core_web_sm')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Load English stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [32]:
# Function for preprocessing the review text
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove any special characters or digits
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenize the text using spaCy
    doc = nlp(text)

    # Remove stopwords, short words, and lemmatize
    tokens = [token.lemma_ for token in doc if token.text not in stop_words and len(token.text) > 2]

    # Join the tokens back into a single string
    return ' '.join(tokens)

In [33]:
# Apply preprocessing to the review_cleaned_text column
df['review_preprocessed_text'] = df['review_cleaned_text'].apply(preprocess_text)

In [34]:
reviews = df.review_preprocessed_text

In [35]:
df.head(8)

,name,city,Business_stars,review_count,is_open,categories,attributes,review_stars,review_useful,review_funny,...,tip_compliment_count,tip_text,tip_cleaned_text,tip_preprocessed_text,year,month,review_sentiment,date,season,sentiment_label
0,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,...,0.0,"Started with a few appetizers - all good, thou...",started with a few appetizers all good though...,startappetizer goodindianpancakestandmaincours...,2012.0,11.0,0.415278,2012-11-01,Rainy,positive
1,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,...,0.0,Awesome!!!,awesome,awesome,2013.0,12.0,0.415278,2013-12-01,Winter,positive
2,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,...,0.0,Good food. There were things I liked and disli...,good food there were things i liked and disliked,goodfoodthinglikedislike,2013.0,3.0,0.415278,2013-03-01,Spring,positive
3,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,...,0.0,Great pad Thai! Great staff and very hospitabl...,great pad thai great staff and very hospitable...,greatpadthaigreatstaffhospitableniceenvironmen...,2012.0,8.0,0.415278,2012-08-01,Summer,positive
4,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,...,0.0,excellent pad thai! love this place. super fas...,excellent pad thai love this place super fast ...,excellentpadthailoveplacesuperfastfriendlyservice,2019.0,7.0,0.415278,2019-07-01,Summer,positive
5,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",3.0,1.0,0.0,...,0.0,"Started with a few appetizers - all good, thou...",started with a few appetizers all good though...,startappetizer goodindianpancakestandmaincours...,2012.0,11.0,0.409174,2012-11-01,Rainy,positive
6,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",3.0,1.0,0.0,...,0.0,Awesome!!!,awesome,awesome,2013.0,12.0,0.409174,2013-12-01,Winter,positive
7,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",3.0,1.0,0.0,...,0.0,Good food. There were things I liked and disli...,good food there were things i liked and disliked,goodfoodthinglikedislike,2013.0,3.0,0.409174,2013-03-01,Spring,positive


In [36]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# TF-IDF Vectorization of review texts
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(reviews)

# Fit the LDA model
lda = LatentDirichletAllocation(n_components=3, random_state=42)  # n_components: number of topics
lda.fit(X)

# Display the top words for each topic
n_words = 10  # Top 10 words per topic
feature_names = np.array(vectorizer.get_feature_names_out())

for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic {topic_idx + 1}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_words - 1:-1]]))

Topic 1:
food place good time great service come make say restaurant
Topic 2:
good order food delicious come like place chicken great fry
Topic 3:
great good oyster food love sandwich place beer service crab


In [37]:
pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.6 MB/s eta 0:00:00


In [38]:
from bertopic import BERTopic
from umap import UMAP

# Set up UMAP with the n_neighbors parameter
umap_model = UMAP(n_neighbors=15)  # Adjust n_neighbors as needed

# Initialize the BERTopic model
topic_model = BERTopic(language="english", umap_model=umap_model, top_n_words=3)

# Fit the model
topics, probs = topic_model.fit_transform(reviews)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:01<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [39]:
# Get a list of valid topic indices (excluding the -1 label for "no topic")
valid_topics = [i for i in set(topics) if i != -1]

# Extract and display the top words for each valid topic
for i in valid_topics:
    # Get the words from the topic, ignoring the scores
    top_words = [word for word, _ in topic_model.get_topic(i)]
    print(f"Topic {i}: {', '.join(top_words)}")

Topic 0: cream, ice, honeycomb
Topic 1: manager, pay, server
Topic 2: drago, charbroile, poboy
Topic 3: dan, macadamia, jordan
Topic 4: okra, cheese, mac
Topic 5: dan, pepper, oil
Topic 6: muffuletta, winner, choco
Topic 7: crust, pizza, pizzeria
Topic 8: coffee, berry, pastry
Topic 9: prompter, slow, lounge
Topic 10: kebab, freebie, foul
Topic 11: admire, architecture, handlebar
Topic 12: overachieve, nonvegetarian, midwest
Topic 13: brunch, latte, sunrise
Topic 14: repeatedly, reputation, boathouse
Topic 15: tampa, bay, cappys
Topic 16: runner, server, slider
Topic 17: thai, gluten, tortilla
Topic 18: bib, recommendedseriously, crabclaw
Topic 19: art, chalk, floor
Topic 20: scott, southwest, job
Topic 21: seasoning, soft, standard
Topic 22: celebration, patio, perfection
Topic 23: beet, tenderloin, urban
Topic 24: conference, paella, mojito
Topic 25: creamy, foodgasm, matthew
Topic 26: pastry, coffee, breakfast
Topic 27: chill, tony, hazelnut
Topic 28: vegan, sprout, shortcake
Topic 

In [40]:
df['Topic'] = topics

# You can use the top words or manually map labels if needed
topic_labels = {}

# Extract the top words for each valid topic and create labels
for i in valid_topics:
    top_words = [word for word, _ in topic_model.get_topic(i)]
    topic_labels[i] = ', '.join(top_words)  # Assign the top words as the label

# Map the topic numbers to their labels and add them to the DataFrame
df['Topic Label'] = df['Topic'].map(topic_labels).fillna("No Topic")

In [41]:
df.drop('Topic',axis=1,inplace=True)

In [42]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, pipeline

# Load the pre-trained model and tokenizer from Hugging Face
tokenizer = RobertaTokenizer.from_pretrained('SamLowe/roberta-base-go_emotions')
model = RobertaForSequenceClassification.from_pretrained('SamLowe/roberta-base-go_emotions')

# Create a Hugging Face pipeline for emotion detection
emotion_model = pipeline('text-classification', model=model, tokenizer=tokenizer,device=0,truncation=True)


tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [43]:
# Function to detect emotion
def detect_emotion(review_text):
    result = emotion_model(review_text)
    predicted_class = result[0]['label']
    return predicted_class

In [44]:
# Apply the emotion detection to the 'review_cleaned_text' column
df['Emotion'] = df['review_cleaned_text'].apply(detect_emotion)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [45]:

# Display the updated DataFrame with emotion labels
df[['review_cleaned_text', 'Emotion']]

,review_cleaned_text,Emotion
0,good ambience\n\ngood service\n\nordered \n\nf...,admiration
1,good ambience\n\ngood service\n\nordered \n\nf...,admiration
2,good ambience\n\ngood service\n\nordered \n\nf...,admiration
3,good ambience\n\ngood service\n\nordered \n\nf...,admiration
4,good ambience\n\ngood service\n\nordered \n\nf...,admiration
...,...,...
10335,i ate here a few times before and was very ple...,admiration
10336,best food ever have been trying hard to find ...,admiration
10337,best food ever have been trying hard to find ...,admiration
10338,awesome dive bar citywides here looooove love...,love


In [46]:
df['Emotion'].value_counts()

,count
Emotion,
admiration,5681
disappointment,812
neutral,782
love,763
joy,621
approval,322
disapproval,243
annoyance,208
gratitude,175


In [47]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [56]:
import openai

In [57]:
#set up the api key
openai.api_key="(Your Secret Key)"

In [58]:
def generate_response(review_text, sentiment, topic):
    # Define the conversation with system, user, and assistant roles
    messages = [
        {"role": "system", "content": "You are a helpful customer service assistant."},
        {"role": "user", "content": f"Review: {review_text}\nSentiment: {sentiment}\nTopic: {topic}"},
    ]

    # Make the API request using the chat model
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Using the chat model
        messages=messages,  # Pass the conversation history
        max_tokens=100,  # Limit the response length
        temperature=0.7,  # Control creativity in response generation
        n=1  # Generate 1 response
    )

    return response.choices[0].message["content"].strip()

In [59]:
df

,name,city,Business_stars,review_count,is_open,categories,attributes,review_stars,review_useful,review_funny,...,tip_cleaned_text,tip_preprocessed_text,year,month,review_sentiment,date,season,sentiment_label,Topic Label,Emotion
0,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,...,started with a few appetizers all good though...,startappetizer goodindianpancakestandmaincours...,2012.0,11.0,0.415278,2012-11-01,Rainy,positive,"resturant, refund, puerto",admiration
1,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,...,awesome,awesome,2013.0,12.0,0.415278,2013-12-01,Winter,positive,"resturant, refund, puerto",admiration
2,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,...,good food there were things i liked and disliked,goodfoodthinglikedislike,2013.0,3.0,0.415278,2013-03-01,Spring,positive,"resturant, refund, puerto",admiration
3,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,...,great pad thai great staff and very hospitable...,greatpadthaigreatstaffhospitableniceenvironmen...,2012.0,8.0,0.415278,2012-08-01,Summer,positive,"resturant, refund, puerto",admiration
4,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,...,excellent pad thai love this place super fast ...,excellentpadthailoveplacesuperfastfriendlyservice,2019.0,7.0,0.415278,2019-07-01,Summer,positive,"resturant, refund, puerto",admiration
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10335,The Cheesecake Factory - Metairie,Metairie,2.5,426.0,1.0,"['Food', 'Restaurants', 'Beer', 'Wine & Spirit...","{'RestaurantsGoodForGroups': 'True', 'GoodForM...",5.0,1.0,1.0,...,i love the food the customer service is great,lovefoodcustomerservicegreat,2017.0,3.0,0.294935,2017-03-01,Spring,positive,"gyro, greek, yat",admiration
10336,Chava's Mexican Restaurant,Edwardsville,3.5,98.0,1.0,"['Restaurants', 'Mexican']","{'RestaurantsTakeOut': 'True', 'Caters': 'True...",5.0,1.0,0.0,...,the fried tacos are excellent great food and ...,friedtacoexcellent greatfoodservicelovely,2014.0,7.0,0.276667,2014-07-01,Summer,positive,"eggbacon, mimosa, queso",admiration
10337,Chava's Mexican Restaurant,Edwardsville,3.5,98.0,1.0,"['Restaurants', 'Mexican']","{'RestaurantsTakeOut': 'True', 'Caters': 'True...",5.0,1.0,0.0,...,i dont do fish tacos ever my husband loves ch...,notfishtaco husbandlovechavasfishtacokeeptrytr...,2014.0,3.0,0.276667,2014-03-01,Spring,positive,"eggbacon, mimosa, queso",admiration
10338,Queen Sheba,Philadelphia,4.0,64.0,1.0,"['Nightlife', 'Restaurants', 'Sports Bars', 'B...","{'BusinessParking': ""{'garage': False, 'street...",4.0,0.0,0.0,...,ask about the old wagon wheel,askoldwagonwheel,2012.0,2.0,0.203968,2012-02-01,Winter,positive,No Topic,love


In [66]:
!pip install openai==0.28 --force-reinstall

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.7 MB/s eta 0:00:00
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━

In [67]:
import openai

def generate_response(review_text, sentiment, topic):
    """
    Generates a response using the OpenAI API based on review text, sentiment, and topic.

    Args:
        review_text (str): The text of the review.
        sentiment (str): The sentiment of the review (e.g., "positive", "negative").
        topic (str): The topic of the review (e.g., "product quality", "customer service").

    Returns:
        str: The generated response from the OpenAI API.
    """
    # Define the conversation with system, user, and assistant roles
    messages = [
        {"role": "system", "content": "You are a helpful customer service assistant."},
        {"role": "user", "content": f"Review: {review_text}\nSentiment: {sentiment}\nTopic: {topic}"},
    ]

    # Make the API request using the chat model (compatible with openai==0.28)
    response = openai.Completion.create(
        engine="text-davinci-003",  # or another suitable engine
        prompt="\n".join([f"{m['role']}: {m['content']}" for m in messages]),
        max_tokens=100,  # Limit the response length
        temperature=0.7,  # Control creativity in response generation
        n=1  # Generate 1 response
    )

    return response.choices[0].text.strip()  # Access the response text

In [68]:
import openai


openai.api_key = ""

# Generate a response using the new API
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",  # or "gpt-4"
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Analyze sentiment for the following text:\n{df['review_cleaned_text'][8]}\n"
                                     f"Sentiment label: {df['sentiment_label'][8]}\n"
                                     f"Topic label: {df['Topic Label'][8]}"}
    ]
)

# Extract the assistant's reply
print(response['choices'][0]['message']['content'])


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [62]:
response = generate_response(df['review_cleaned_text'][8], df['sentiment_label'][8], df['Topic Label'][8])


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [69]:
df['review_cleaned_text'][8]

'this place was good but was nothing amazing at least from what i ordered  it was my first visit this afternoon  i was here for restaurant week\n\nmy first course was the wonton soup  good but nothing special  i then received the chili aioli rock shrimp  this was a tasty piece and was presented nicely\n\nthe main entree was the cumin beef  i had no issue with the quality and it was very good but i felt the dish did not distinguish itself from what you could get in a chinese takeout spot  everything was cooked well and you could definitely taste the spice when you first sample it  it was a good dish and i finished it but it was nothing that i was really impressed with  i finished off with a tiramisu and this was also good but nothing fantastic  \n\nthe one nice thing with this place is they brew there own tea and so you wont be getting the lipton stuff when order one  \n\nmy rating does not reflect any negative experiences with this venue  i just was not too impressed with the flavor of

In [70]:
response_templates = {
    "positive": {
        "food quality": "Thank you for your positive feedback about the food! We're thrilled you enjoyed it.",
        "service": "We're so happy to hear you had a great experience with our service!",
        "ambiance": "Thank you for your kind words about the ambiance. We're glad you felt comfortable here!"
    },
    "negative": {
        "food quality": "We're sorry to hear the food didn't meet your expectations. We will work on improving our menu.",
        "service": "We apologize for the slow service and will ensure better attention in the future.",
        "noise": "We're sorry to hear that the noise affected your experience. We'll consider making improvements to the atmosphere."
    }
}

In [71]:
def generate_template_response(review,sentiment, topic):
    # Select appropriate template based on sentiment and topic
    try:
        return response_templates[sentiment][topic]
    except KeyError:
        return generate_response(review, sentiment, topic)